# Genome Mining Script
##### Original Author: Chun Yin Larry So | Python Interpretation: Nathan Alam
[Github repository](https://github.com/nathanalam/genomemining)

The purpose of this notebook is to provide scripts for reading bacterial genomes in search of regular expressions which seem to match a String corresponding to a candidate coding for a Lasso peptide.

## Links to relevant publications:
- [Genome mining for lasso peptides: past, present, and future](https://link.springer.com/article/10.1007/s10295-019-02197-z)
- [Prospecting Genomes for Lasso Peptides](https://www.ncbi.nlm.nih.gov/pubmed/24142336)

### Comments from Perl Script
- In this version each time MAST needs to be run on a putative maturation enzyme in a particular genome, a lookup in the database is performed to check if this protein has been analyzed by MAST before and if it has then use the values from before
- In this version traseq will be used instead of getorf for finding precursors
- getorf is still used for finding neighbors
- Like v4 except that multiple proteins of the same sequence don't cause wrong locations of maturation enzymes to be reported
- Another change is that the pattern needs to be adjusted to [MVL] instead of ^ in the beginning
- This version fixes the problem of having a useless %AME hash and also of erasing sequences from %AME_scores
- Precursor pattern is output into the log file
- ORF searching behavior is changed from stop-to-stop to [MVL]-to-stop
- Clusters of precursors are saved in clusters.txt
- **Warning**: transeq doesn't label the -1, -2, -3 frames sequentially. Sometimes it is -1, -3, -2, sometimes some other combination. This means the precursor locations on the reverse strand are off by one sometimes
- Take note that rank_hits expects 4 motifs for the B enzyme and 3 motifs for the C enzyme. Adjust accordingly.

In [14]:
import re
import sys
import os
import json
import requests

### THE Pattern

This is the pattern that we're using to identify lasso proteins. TODO - Use Machine Learning to adjust the pattern to maximize the number of valid lasso proteins.

In [15]:
PATTERN = '^M.{15,45}T..{6,8}[DE].{5,30}$'
# PATTERN = 'CC.CGCCC...TGGC.'
# PATTERN = '.*'

### FASTA Function

Define a function that takes as input the relative path of a FASTA formatted text file, return an object that contains a list of sequence objects. Each sequence object has a description field ["description"] and a sequence field ["sequence"].

From http://www.csbio.sjtu.edu.cn/bioinf/virus-multi/example.htm, specification of a FASTA formatted file:
- The first line of each query protein input format must begin with a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier and description of the sequence, but both are optional.
- The sequence (in single-character code) begins in a different line and ends if another line starting with a ">" appears, which indicates the start of another query protein.

In [3]:
def readFASTA(name, cleanspace = 0):
    descriptions = []
    sequences = []
    sequenceList = []
    tempSequences = []     
        
    with open(name) as file:
        count = -1
        for line in file:
            
            if(line[0] == '>'):
                # if begins with a >, then a description
                descriptions.append(line[1:].replace('\n', ''))
                count += 1
                # skip the first time
                if count > 0 :
                    # combine the tempSequences into a single string and
                    # add it to sequences
                    newSequence = ' '.join(tempSequences)
                    # now remove all of the whitespaces
                    newSequence = newSequence.replace(' ', '')
                    newSequence = newSequence.replace('\n', '')
                    
                    sequences.append(newSequence)
                    # refresh the tempSequence list
                    tempSequences = []
                    
                    sequenceList.append({
                        "description": descriptions[count - 1],
                        "sequence": sequences[count - 1]
                    })
            else:
                tempSequences.append(line)
                
        # combine the tempSequences into a single string and
        # add it to sequences
        newSequence = ' '.join(tempSequences)
        # now remove all of the whitespaces
        newSequence = newSequence.replace(' ', '')
        newSequence = newSequence.replace('\n', '')

        sequences.append(newSequence)
        # refresh the tempSequence list
        tempSequences = []
        
        sequenceList.append({
            "description": descriptions[count],
            "sequence": sequences[count]
        })
                
                
    if len(descriptions) != len(sequences):
        print("ERROR: Number of descriptions does not match number of sequences")
        print("Number of descriptions: " + str(len(descriptions)))
        print("Number of sequences: " + str(len(sequences)))
        sys.exit(1);
        
    print("Read " + str(count + 1) + " objects from FASTA file " + name)
        
    return sequenceList
        


### Obtain Amino acid sequence directories

Begin by getting a list of all of the genomes available in the genomes folder alongside this script.

In [4]:
ALLDIRNAMES = []
for dirname in os.listdir("genomes"):
    ALLDIRNAMES.append("genomes/" + dirname)

For the fna files, we need to convert them to amino acid sequences (or faa files). We do this using [Emboss Transeq](https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/EMBOSS+transeq+Help+and+Documentation#EMBOSStranseqHelpandDocumentation-Reference), and save the output amino acid sequences into this genome file

In [5]:
import time
url = "https://www.ebi.ac.uk/Tools/services/rest/emboss_transeq"

## An adapter function for the Emboss transeq, takes in a DNA sequence and returns a list of protein sequences
def emboss_transeq(DNAseq):
    payload = "email=nalam@princeton.edu&sequence=" + DNAseq + "&codontable=11&frame=6"
    headers = {
        'Content-Type': "application/x-www-form-urlencoded",
        'Accept': "text/plain",
        'User-Agent': "PostmanRuntime/7.13.0",
        'Cache-Control': "no-cache",
        'Postman-Token': "e19f3174-9eb3-43e2-ade4-c73702e88fa0,43c05bc6-a184-4288-8e52-bef78f1f9a9b",
        'Host': "www.ebi.ac.uk",
        'accept-encoding': "gzip, deflate",
        'content-length': "282",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }
    abbrevlen = 25 
    if (25 > len(DNAseq)):
        abbrevlen = len(DNAseq) 
    print("Beginning training job for " + DNAseq[:abbrevlen] + "...")
    
    response = requests.request("POST", url + "/run", data=payload, headers=headers)
    jobID = response.text
    print("Job begun with jobID " + jobID)
    
    stat = "RUNNING"
    
    while(stat != "FINISHED"):
        response = requests.request("GET", url + "/status/" + jobID, headers=headers)
        print(response.text)
        stat = response.text
        time.sleep(1)
    
    
    if stat == "FINISHED":
        print("Requesting results for " + jobID)
        response = requests.request("GET", url + "/result/" + jobID + "/out", headers=headers)
        print("Writing results to temp.faa")
        with open('temp.faa', 'w') as outfile:
            outfile.write(response.text)
        print("Reading results with FASTA reader from temp.faa")
        seqList = readFASTA("temp.faa")
        
        print("Finished reading, now removing temp.faa")
        os.remove("temp.faa")
        
        returnList = []
        for obj in seqList:
            returnList.append(obj["sequence"])
        return returnList

    else:
        print("Something went horribly wrong...")
        return []
    
#     AAList = ["", "", "", "", "", ""]
#     codon = ""
#     for e in range(0, 6):
#         AAList[e] = 'A' * int(len(DNAseq) / 3)
    

In [6]:
emboss_transeq("AUG")

Beginning training job for AUG...
Job begun with jobID emboss_transeq-R20190930-182012-0830-7651441-p2m
RUNNING
FINISHED
Requesting results for emboss_transeq-R20190930-182012-0830-7651441-p2m
Writing results to temp.faa
Reading results with FASTA reader from temp.faa
Read 6 objects from FASTA file temp.faa
Finished reading, now removing temp.faa


['M', 'X', 'X', 'H', 'X', 'X']

In [7]:
for dirname in ALLDIRNAMES:
    if((dirname[len(dirname) - 3:] == "fna") and not (dirname[:len(dirname) - 3] + "faa") in ALLDIRNAMES):
        print("Opening up " + dirname + " and converting into peptide sequences...")
        DNAseqs = []
        seqDescriptions = []
        for fastaobj in readFASTA(dirname):
            DNAseqs.append(fastaobj["sequence"])
            seqDescriptions.append(fastaobj["description"])
            
        entries = []
        for i in range(0, len(DNAseqs)):
            print("converting " + str(len(DNAseqs[i])) + " base pairs from " + seqDescriptions[i])
            aalist = emboss_transeq(DNAseqs[i])
            print("created " + str(len(aalist)) + " peptide sequences from " + seqDescriptions[i])
            for e in range(0, len(aalist)):
                entries.append({
                    "sequence": aalist[e],
                    "description": str(seqDescriptions[i] + " - peptide " + str(e)) 
                })
        
        print("writing read peptides into '" + dirname[len('genomes/'):len(dirname) - 3] + "faa'")
        with open(dirname[:len(dirname) - 3] + "faa", 'w') as outfile:
            for ent in entries:
                outfile.write("> " + ent["description"] + "\n")
                outfile.write(ent["sequence"] + "\n\n")

Opening up genomes/btha.fna and converting into peptide sequences...
Read 1 objects from FASTA file genomes/btha.fna
converting 37774 base pairs from gi|83716035|ref|NC_007650|pseudocap|177 [Burkholderia thailandensis E264 chromosome II, complete sequence.]
Beginning training job for ATGACTCTCGACGAGATCCGGCAAT...
Job begun with jobID emboss_transeq-R20190930-182130-0850-95857415-p2m
FINISHED
Requesting results for emboss_transeq-R20190930-182130-0850-95857415-p2m
Writing results to temp.faa
Reading results with FASTA reader from temp.faa
Read 6 objects from FASTA file temp.faa
Finished reading, now removing temp.faa
created 6 peptide sequences from gi|83716035|ref|NC_007650|pseudocap|177 [Burkholderia thailandensis E264 chromosome II, complete sequence.]
writing read peptides into 'btha.faa'


Now, we append all of the files ending in a ".faa" translation to an array of files called DIRNAMES

In [8]:
DIRNAMES = []
for dirname in os.listdir("genomes"):
    if(dirname[len(dirname) - 3:] == "faa"):
        DIRNAMES.append("genomes/" + dirname)
        
print(DIRNAMES)

['genomes/btha.faa']


### Pattern Matching
Uses the python regular expression library to determine whether proteins match the pattern sequence. This function takes in an overall sequence of amino acids and determines whether the sequence passes the pattern regular expression. The function returns a list of matched proteins, which have a specific sequence, and stores the overall sequence and the associated description.

In [9]:
def patternMatch(overallSequence, pattern, description):
    matchedProteinList = []
    
    # find all matches in protein that match
    matchIter = re.finditer(pattern, overallSequence)
    done_looping = False
    while not done_looping:
        try:
            match = next(matchIter)
        except StopIteration:
            done_looping = True
        else:
            matchedProteinList.append({
                "description": description,
                "sequence": match.group(0),
                "searchPattern": match.re.pattern,
                "searchRange": match.span(),
                "overallString": match.string
            })
    return matchedProteinList

In [10]:
matchedProteins = []
for filename in DIRNAMES:
    readSequences = readFASTA(filename)
    for seq in readSequences:
        matchedProteins.extend(patternMatch(seq["sequence"], PATTERN, filename + " - " + seq["description"]))
    

print("Found " + str(len(matchedProteins)) + " that satisfy the pattern: " + PATTERN)
# for match in matchedProteins:
#     print(match["sequence"] + ", found in range " + str(match["searchRange"]))
#     print("description: " + match["description"])

for match in matchedProteins:
    print(match["description"])

Read 6 objects from FASTA file genomes/btha.faa
Found 0 that satisfy the pattern: ^M.{15,45}T..{6,8}[DE].{5,30}$


In [11]:
with open('matches.json', 'w') as outfile:
    json.dump(matchedProteins, outfile)

In [13]:
lassopeptides = []
with open('matches.json', 'r') as storedfile:
    lassopeptides = json.loads(storedfile.read())

print(lassopeptides)

[]
